While each Kind in Kubernetes is unique, they all follow a similar structure. Each Kubernetes object satisfies a *runtime.Object* interface in Go which has the following structure.

In [ ]:
type Object interface {
    GetObjectKind() schema.ObjectKind
    
    type ObjectKind interface {
        SetGroupVersionKind(kind GroupVersionKind)
        GroupVersionKind() GroupVersionKind
    }
}

This allows for the object to be deep copied and have its GroupVersionKind both returned to the user and set. Beyond those interfaces, TypeMeta provides how the object is represented through the API and looks like so.

In [ ]:
type TypeMeta struct {
    Kind string `json: "kind, omitempty" protobuf: "bytes, 1, opt, name=kind"`
    APIVersion string `json: "apiversion, omitempty"`
}

// an actual kind definition in Go would look something like the following

type CustomResource struct {
    metav1.TypeMeta `json: "inline"`
    metav1.ObjectMeta `json: "metadata, omitempty"`
    Spec CustomResourceSpec `json: "spec, omitempty"`
    Status CustomResourceStatus `json: "status, omitempty"`
}

// you will then have to implement the methods required by the runtime.Object interface for it to be usable
// ObjectMeta is another field that most objects in Kubernetes have, as it implies, it holds the object's metadata
// This is the stuff you'd find under metadata

type ObjectMeta struct {
    Name string `json:"name,omitempty"`
    Namespace string `json:"namespace,omitempty"`
    UID types.UID `json:"uid,omitempty"`
    ResourceVersion string `json:"resourceVersion,omitempty"`
    CreationTimestamp Time `json:"creationTimestamp,omitempty"`
    DeletionTimestamp *Time `json:"deletionTimestamp,omitempty"`
    Labels map[string]string `json:"labels,omitempty"`
    Annotations map[string]string `json:"annotations,omitempty"`
    ...
}

The spec field is what the user desires of the object and its up to that resource's controller to make that a reality. A controller or operator is usually the thing that fills out an object's status field reflecting the actual state of the object in the cluster. There is also a difference between chaning the status and spec of an object. Usually when you want to patch an object it goes to the object's endpoint like */apis/apps/v1/namespaces/{namespace}/customresource/{CRname}* however when you change the object, its status also has to reflect that and can be changed at the same endpoint but just by adding */status* to the end. 

<h2>Informers and Caching</h2>

When it comes to your controllers, they update and review your resources, fbut depending on how many resources a controller is looking after and the possibility that you can have more than 1 controller leads to the problem of those controllers putting strain on the API server. This is where informers (helpers that watch for events on objects you're monitoring) and caches (in memory representation of those objects you got from the informers) come in. A more advanced version of informers are the *shared informer factories* which have multiple informers share 1 cache. This is useful if you have multiple controllers but don't want to strain the API server too much. In essence, USE THE SHARED INFORMERS INSTEAD OF MANUALLY CREATING THEM!!! Since your resources are also in memory, nothing will happen if you try to update the informer's in-memory rep of the object you're trying to modify!! This will only change the *informer's* version of the object and not that of the actual object in the cluster! This can lead controllers to do wacky things so NEVER mutate the informer's objects!! **In essence, use the client to write changes to resources so that they are actually reflected in the cluster.** A good rule of thumb to follow is asking yourself "Who owns this object?" clients own the objects they get from the API server but not the ones that the informer gets from Listers; you'll have to deep copy those objects if you want to mutate and write them to the API server. 

<h2>Workqueues</h2>

This is the third (client, informer, workqueue) main building block of a controller. The workqueues are formally priority queues, i.e. a FIFO structure that puts priority on the elements added to it. A workqueue interface would look like this

In [ ]:
type Interface interface {
    Add(item interface{}) // remember that a type of interface{} means ANY type is accepted
    Len() int
    Get() (item interface{}, shutdown bool)
    Done(item interface{})
    Shutdown()
    ShuttingDown() bool
}

Look at page 83 for an explanation on what each method does. This interface is then used within another generic interface which you have two options between.

type DelayingInterface interface {
    Interface
    AddAfter(item interface{}, duration time.Duration)
}

type RateLimitingInterface interface {
    DelayingInterface
    AddRateLimited(item interface{})
    Forget(item interface{})
    NumRequeues(item interface{})
}

<h2>API Machinery and Types</h2>

In API Machinery (APIM) they don't use kinds as you would in Go, but Kinds as you would in Kubernetes. Each kind is divided an API group and a version i.e. deployments are part of the *apps* API group and currently have a version of 1. This identifier is reffered to as the GroupVersionKind (GVK) and is directly correspondent to ONE Go type. However, Go types can belong to many different GVKs such as the *Status* types which are required by every Kind. GroupVersionResources (GVRs) are also grouped and versioned but they point to specific endpoints in a cluster. Then we have RESTMappers which take a GVK and translate it to a GVR. For right now, we just have to know that these exist so we're not confused when we see them later when the book goes over an actual controller implementation. 

<h2>Scheme</h2>

This is the last and arguably most important thing to understand about how your controller and Go types integrate with Kubernetes (in my opinion). The main purpose of the scheme is to *map Golang types with possible correspondent GVKs*. The scheme uses reflection to add Golang types to the world of Kubernetes but can only do so with known Go types i.e. you have to add your type to the scheme if you want it to be usable in Kubernetes. The important thing to remember about this APIM deep dive is that Go Structs -(Scheme)-> GVKs -(RESTMapping)-> GVRs -(client)-> HTTP Path. 